# Residual Networks

**Residual Networks** allow us to train relatively deeper networks than were previously practically feasible. Here we will use **Keras**.

In [5]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


## 1. The Problem of Very Deep Neural Networks

**Pros of Deep NNs**
- can represent very complex functions 
- can learn features at different levels of abstraction, from edges to very complex features 


**Cons of Deep NNs**
- vanishing gradients: gradient signal goes to zero quickly, making gradient descent prohibitively slow
- the magnitude (norm) of the gradient for the shallower layers decrease to zero very rapidly 

## 2. Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers. By stacking ResNet blocks, we can form very deep networks. ResNet blocks make it very easy for a block to learn an identity function, meaning that you can stack on additional ResNet blocks without harming training set performance.  

Two main types of blocks are used in a ResNet: the **identity block** and the **convolutional block**. This depends mainly on whether the input/output dimensions are the same or different.

### 2.1 - The Identity Block

The **identity block** is the standard block used in ResNets, and is used when the input activation ($a^{[l]}$) has the same dimension as the output activation ($a^{[l+2]}$). Below, the upper path is the shortcut path, while the lower path is the main path. To speed up training, the BatchNorm step is added. 

```
Arguments:
X = (m, n_H_prev, n_W_prev, n_C_prev)
f = the shape of the middle CONV's window for the main path
filters = list of the numbers of filters in the CONV layers of the main path
stage, block = used to name layers, depending on positions in network    

Returns:
X = output of the identity block = (n_H, n_W, n_C)
```

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'></u><font color='purple'> **Identity block.** Skip connection "skips over" 3 layers.</center></caption>

In [6]:
def identity_block(X, f, filters, stage, block):
    
    # define name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters      # retrieve filters 
    X_shortcut = X            # save the input value to add back to the main path
    
    # first component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', 
               name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
        
    # second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1,1), padding='same', 
               name=conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', 
               name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])         # add shortcut value to main path
    X = Activation('relu')(X)          # pass it through RELU
        
    return X

### 2.2 - The Convolutional Block

The "convolutional block" is used when the input and output dimensions don't match. The difference with the identity block is that there is a CONV2D layer in the shortcut path. This CONV2D layer is used to resize input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. This layer does not use any non-linear activation function, but rather applies a learned linear function that reduces the dimension of the input.

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'></u><font color='purple'> **Convolutional block** </center></caption> 

In [7]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    
    # define name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters      # retrieve filters
    X_shortcut = X            # save the input value

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(F2, (f, f), strides = (1,1), name = conv_name_base + '2b', 
               padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', 
               padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', 
                        padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])          # add shortcut value to main path
    X = Activation('relu')(X)           # pass it through RELU
    
    return X

## 3 - Building a ResNet Model (50 layers)

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'></u><font color='purple'> **ResNet-50 model** </center></caption>

In [8]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    
    X_input = Input(input_shape)               # define input with shape input_shape
    X = ZeroPadding2D((3, 3))(X_input)         # zero-padding
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), name='avg_pool')(X)
    
    # Output Layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [9]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Load the SIGNS dataset

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [11]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

Epoch 1/2
1080/1080 [==============================] - 233s - loss: 3.0649 - acc: 0.2620   
Epoch 2/2
1080/1080 [==============================] - 240s - loss: 1.9921 - acc: 0.3750   


In [12]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 7s     
Loss = 2.12532908122
Test Accuracy = 0.5


In [13]:
model = load_model('ResNet50.h5') 

In [14]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 7s     
Loss = 0.255690792203
Test Accuracy = 0.91666667064
